In [1]:
import numpy as np
import tensorflow as tf
import random
import math
import h5py
import gc
import sys

In [2]:
def acc(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 2).T == labels) / predictions.shape[1] / predictions.shape[0])

In [3]:
def mean(numbers):
    return float(sum(numbers)) / max(len(numbers), 1)

In [6]:
hdf_file = 'datasets/pickles/SVHN_multi_box.hdf5'

hdf = h5py.File(hdf_file,'r')
train_dataset = hdf['train_images'][:]
train_labels = hdf['train_labels'][:]
test_dataset = hdf['test_images'][:]
test_labels = hdf['test_labels'][:]
valid_dataset = hdf['valid_images'][:]
valid_labels = hdf['valid_labels'][:]
            
hdf.close()    

print('Training set', train_dataset.shape, train_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)

Training set (230070, 32, 96, 1) (230070, 6)
Test set (13068, 32, 96, 1) (13068, 6)
Validation set (5684, 32, 96, 1) (5684, 6)


In [7]:
train_dataset = train_dataset.astype(np.float32)
test_dataset = test_dataset.astype(np.float32)
valid_dataset = valid_dataset.astype(np.float32)

train_labels = train_labels.astype(np.int32)
test_labels = test_labels.astype(np.int32)
valid_labels = valid_labels.astype(np.int32)

In [8]:
model_to_save = "saved_models/combined/box_on_mnist/CNN_SVHN_Box_on_Mnist.ckpt"
saved_mnist_model = "saved_models/mnist/CNN_SVHN_Mnist.ckpt"

In [9]:
graph_svhn = tf.Graph()

with graph_svhn.as_default():
    HEIGHT = 32
    WIDTH = 32*3
    
    X = tf.placeholder(tf.float32, [None, HEIGHT, WIDTH, 1])
    Y_ = tf.placeholder(tf.int32, [None, 6])
    
    # Learning Rate - alpha
    alpha = tf.placeholder(tf.float32)
    
    # Dropout Probablity
    pkeep = tf.placeholder(tf.float32)
    
    # 5 Layers and their no of neurons
    # 3 Convolutional Layers and a fully connected layer
    K = 6     # First Conv Layer with depth 6
    L = 12     # Second Conv Layer with depth 12
    M = 24    # Third Conv layer with depth 24
    N = 200   # Fourth Fully Connected layer with 200 neurons
    # Last one will be softmax layer with 10 output channels
    
    W1 = tf.Variable(tf.truncated_normal([6, 6, 1, K], stddev=0.1), name="W1")    # 6x6 patch, 1 input channel, K output channels
    B1 = tf.Variable(tf.constant(0.1, tf.float32, [K]), name="B1")
    
    W2 = tf.Variable(tf.truncated_normal([5, 5, K, L], stddev=0.1), name="W2")
    B2 = tf.Variable(tf.constant(0.1, tf.float32, [L]), name="B2")
    
    W3 = tf.Variable(tf.truncated_normal([4, 4, L, M], stddev=0.1), name="W3")
    B3 = tf.Variable(tf.constant(0.1, tf.float32, [M]), name="B3")
    
    W5_1 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1), name="W5_1")
    B5_1 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_1")
    
    W5_2 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1), name="W5_2")
    B5_2 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_2")
    
    W5_3 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1), name="W5_3")
    B5_3 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_3")
    
    W5_4 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1), name="W5_4")
    B5_4 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_4")
    
    W5_5 = tf.Variable(tf.truncated_normal([N, 11], stddev=0.1), name="W5_5")
    B5_5 = tf.Variable(tf.constant(0.1, tf.float32, [11]), name="B5_5")
    
    # Model
    stride = 1  # output is 32x96
    Y1 = tf.nn.relu(tf.nn.conv2d(X, W1, strides=[1, stride, stride, 1], padding='SAME') + B1)
    
    stride = 2  # output is 16x48
    Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=[1, stride, stride, 1], padding='SAME') + B2)
    
    stride = 2  # output is 8x24
    Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=[1, stride, stride, 1], padding='SAME') + B3)

    # reshape the output from the third convolution for the fully connected layer
    shape = Y3.get_shape().as_list()
    YY = tf.reshape(Y3, shape=[-1, shape[1] * shape[2] * shape[3]])
    
    W4 = tf.Variable(tf.truncated_normal([shape[1] * shape[2] * shape[3], N], stddev=0.1), name="W4")
    B4 = tf.Variable(tf.constant(0.1, tf.float32, [N]), name="B4")

    Y4 = tf.sigmoid(tf.matmul(YY, W4) + B4)
    YY4 = tf.nn.dropout(Y4, pkeep)
    
    Ylogits_1 = tf.matmul(YY4, W5_1) + B5_1
    Ylogits_2 = tf.matmul(YY4, W5_2) + B5_2
    Ylogits_3 = tf.matmul(YY4, W5_3) + B5_3
    Ylogits_4 = tf.matmul(YY4, W5_4) + B5_4
    Ylogits_5 = tf.matmul(YY4, W5_5) + B5_5   
    ## ('Ylogits_1 shape : ', [None, 11])
    
    Y_1 = tf.nn.softmax(Ylogits_1)
    Y_2 = tf.nn.softmax(Ylogits_2)
    Y_3 = tf.nn.softmax(Ylogits_3)
    Y_4 = tf.nn.softmax(Ylogits_4)
    Y_5 = tf.nn.softmax(Ylogits_5)
   
    cross_entropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_1, Y_[:,1])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_2, Y_[:,2])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_3, Y_[:,3])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_4, Y_[:,4])) +\
    tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(Ylogits_5, Y_[:,5]))

    train_prediction = tf.pack([Y_1, Y_2, Y_3, Y_4, Y_5])
    
    train_step = tf.train.AdamOptimizer(alpha).minimize(cross_entropy)
    
    W_s = tf.pack([tf.reduce_max(tf.abs(W1)),tf.reduce_max(tf.abs(W2)),tf.reduce_max(tf.abs(W3)),tf.reduce_max(tf.abs(W4))])
    b_s = tf.pack([tf.reduce_max(tf.abs(B1)),tf.reduce_max(tf.abs(B2)),tf.reduce_max(tf.abs(B3)),tf.reduce_max(tf.abs(B4))])
    
    model_saver = tf.train.Saver()

In [10]:
train_data = train_dataset
label_data = train_labels
print('train : ', train_data.shape, '  test : ', label_data.shape)

train :  (230070, 32, 96, 1)   test :  (230070, 6)


In [12]:
num_steps_1 = 30001
batch_size = 128

with tf.Session(graph=graph_svhn) as session:
#     tf.global_variables_initializer().run()
    model_saver.restore(session, saved_mnist_model)
    print('Initialized')
    
    for step in range(num_steps_1):
        #  learning rate decay
        max_learning_rate = 0.0005
        min_learning_rate = 0.0001

        decay_speed = 5000.0
        learning_rate = min_learning_rate + (max_learning_rate - min_learning_rate) * math.exp(-step/decay_speed)
        offset = (step * batch_size) % (label_data.shape[0] - batch_size)
        batch_data = train_data[offset:(offset + batch_size), :, :, :]
        batch_labels = label_data[offset:(offset + batch_size), :]
        
        feed_dict = {X : batch_data, Y_ : batch_labels, pkeep : 0.80, alpha : learning_rate}
        _, l, train_pred, W, b = session.run([train_step, cross_entropy, train_prediction, W_s, b_s], feed_dict=feed_dict)
    
        if (step % 500 == 0):
            print('W : ', W)
            print('b : ', b)
            print('Loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % acc(train_pred, batch_labels[:,1:6]))
            print('Learning rate : ', learning_rate)
            print('    ')
            
    print('Training Complete on MNIST Data')
    
    save_path = model_saver.save(session, model_to_save)
    print("Model saved in file: %s" % save_path)

Initialized
W :  [ 0.37272164  0.48718435  0.31338772  0.49764675]
b :  [ 0.18485929  0.1950376   0.4238168   0.19206028]
Loss at step 0: 25.621204
Minibatch accuracy: 21.6%
Learning rate :  0.0005
    
W :  [ 0.26441282  0.43882173  0.32453412  0.45159847]
b :  [ 0.10925158  0.25092804  0.45009527  0.21947894]
Loss at step 500: 5.465186
Minibatch accuracy: 64.7%
Learning rate :  0.00046193496721438383
    
W :  [ 0.28102431  0.42248625  0.33415824  0.45742369]
b :  [ 0.10620536  0.27386266  0.44928122  0.22840211]
Loss at step 1000: 4.387840
Minibatch accuracy: 73.0%
Learning rate :  0.00042749230123119273
    
W :  [ 0.2899048   0.40556544  0.34720343  0.49042353]
b :  [ 0.12619545  0.28770003  0.4479593   0.23237409]
Loss at step 1500: 3.684948
Minibatch accuracy: 77.0%
Learning rate :  0.00039632728827268716
    
W :  [ 0.2954677   0.3880918   0.36182961  0.5223251 ]
b :  [ 0.13295738  0.29047504  0.44519916  0.2308701 ]
Loss at step 2000: 3.584943
Minibatch accuracy: 76.7%
Learnin

In [13]:
with tf.Session(graph=graph_svhn) as session: 
    print('Initialized')
    batch = 1000
    
    test_acc = list()
    print('-------TEST--------')
    test_no = int(test_labels.shape[0] / batch)
    for i in range(test_no - 1):
        model_saver.restore(session, model_to_save)
        data = test_dataset[i*batch:(i+1)*batch]
        labels = test_labels[i*batch:(i+1)*batch]
        
        _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : data, Y_ : labels, pkeep : 1.0, alpha : 0.002})
        accuracy = acc(predictions, labels[:,1:6])
        test_acc.append(accuracy)
        
        print('Test-Accuracy', ' i : ', i)
        print('Test accuracy: ', accuracy)
        print('       ')
        
        
    valid_acc = list()
    print('-----VALIDIDATION------')
    valid_no = int(valid_labels.shape[0] /  batch)
    for i in range(valid_no - 1):
        model_saver.restore(session, "saved_models/box/CNN_SVHN_Box.ckpt")
        data = valid_dataset[i*batch:(i+1)*batch]
        labels = valid_labels[i*batch:(i+1)*batch]
        
        _, l, predictions = session.run([train_step, cross_entropy, train_prediction], feed_dict={X : data, Y_ : labels, pkeep : 1.0, alpha : 0.002})
        accuracy = acc(predictions, labels[:,1:6])
        valid_acc.append(accuracy)
        
        print('Valid-Accuracy', ' i : ', i)
        print('Valid accuracy: ', accuracy)
        print('        ')
        
        
    test_avg = mean(test_acc)
    valid_avg = mean(valid_acc)
    
    print('-----  FINAL  ------')
    print('Final Test Set Accuracy : ',"%.2f" % test_avg)
    print('Final Validation Set Accuracy : ',"%.2f" % valid_avg)

Initialized
-------TEST--------
Test-Accuracy  i :  0
Test accuracy:  93.3
       
Test-Accuracy  i :  1
Test accuracy:  93.5
       
Test-Accuracy  i :  2
Test accuracy:  93.46
       
Test-Accuracy  i :  3
Test accuracy:  93.54
       
Test-Accuracy  i :  4
Test accuracy:  93.62
       
Test-Accuracy  i :  5
Test accuracy:  93.8
       
Test-Accuracy  i :  6
Test accuracy:  93.24
       
Test-Accuracy  i :  7
Test accuracy:  93.5
       
Test-Accuracy  i :  8
Test accuracy:  93.14
       
Test-Accuracy  i :  9
Test accuracy:  93.48
       
Test-Accuracy  i :  10
Test accuracy:  93.34
       
Test-Accuracy  i :  11
Test accuracy:  92.72
       
-----VALIDIDATION------
Valid-Accuracy  i :  0
Valid accuracy:  93.8
        
Valid-Accuracy  i :  1
Valid accuracy:  93.56
        
Valid-Accuracy  i :  2
Valid accuracy:  92.36
        
Valid-Accuracy  i :  3
Valid accuracy:  91.84
        
-----  FINAL  ------
Final Test Set Accuracy :  93.39
Final Validation Set Accuracy :  92.89


In [14]:
train_data = train_dataset
label_data = train_labels
print('train : ', train_data.shape, '  test : ', label_data.shape)

num_steps_1 = 25001
batch_size = 128

with tf.Session(graph=graph_svhn) as session:
    model_saver.restore(session, model_to_save)
    print('Initialized')
    
    W1 = session.run(W1)
    print(W1)

train :  (230070, 32, 96, 1)   test :  (230070, 6)
Initialized
[[[[ -1.50522776e-02  -2.64132079e-02   1.52027056e-01   2.12814603e-02
      1.38093442e-01   5.17821796e-02]]

  [[  4.74680550e-02   2.50265330e-01  -7.72794187e-02  -3.27843428e-02
     -1.74948499e-01   9.23548341e-02]]

  [[  3.13280448e-02   1.62439570e-01  -1.15763634e-01   2.56980397e-02
     -5.72252236e-02  -2.33353879e-02]]

  [[  6.30227774e-02  -3.36757153e-02  -7.43347034e-02  -1.85744002e-01
      1.70991588e-02   3.83118913e-02]]

  [[ -6.02321662e-02   1.58646643e-01  -7.95831308e-02   3.92488986e-02
     -5.92272915e-02  -4.05822210e-02]]

  [[ -1.90515310e-01   1.36464490e-02  -1.43729493e-01   1.98645294e-01
      9.47273299e-02   6.18795007e-02]]]


 [[[ -1.71603207e-02   3.16025503e-02  -8.37688297e-02   1.30627722e-01
     -8.66929814e-02  -2.93678679e-02]]

  [[ -4.22022454e-02  -1.76658720e-01   3.39092501e-02  -7.32576922e-02
     -1.08213857e-01   1.77022755e-01]]

  [[  8.91070664e-02  -6.988737